In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-1p1xqv0w
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-1p1xqv0w
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4293 sha256=5609bfba5493d055501bef8b1f1aa5349c438619c441f969a072c1b8a61f547b
  Stored in directory: /tmp/pip-ephem-wheel-cache-13q4rm1r/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [11]:
%%cu
#include <iostream>
#include <cuda_runtime.h>
#define T_VEC 100
//DEFINICIÓN KERNEL CUDA
__global__ void sumaVectores(int* v1_device, int* v2_device, int*
suma_device) {
printf("Índice de hilo dentro del bloque: %d\n", threadIdx.x);
printf("Índice del bloque: %d\n", blockIdx.x);
printf("Tamaño del bloque (número de hilos): %d\n", blockDim.x);
//al tener 25 hilos se ejecuta 25 veces por bloque entonces lo unico quue hay que hacer es
//hacer un bucle de 4 iteraciones sumandole al id del thread 25* la iteracion asía haciendose solo 100 veces
for (int i = 0; i < 4; i++) {
suma_device[threadIdx.x+(25*i)] = v1_device[threadIdx.x+(25*i)] + v2_device[threadIdx.x+(25*i)];
  printf("I del bucle: %d\n", (threadIdx.x+(25*i)));

}

}
//FUNCIÓN DEL HOST
int main() {
//1 - INICIALIZAR DATOS DEL HOST
int v1_host[T_VEC], v2_host[T_VEC], suma_host[T_VEC];
for (int i = 0; i < T_VEC; i++) {
v1_host[i] = 1;
v2_host[i] = 2;
}
//2 - DECLARAR Y ASIGNAR MEMORIA PARA EL DEVICE
int *v1_device, *v2_device, *suma_device; //punteros al device
cudaMalloc((void**)&v1_device, sizeof(int) * T_VEC); //asignar memoria para el device
cudaMalloc((void**)&v2_device, sizeof(int) * T_VEC);
cudaMalloc((void**)&suma_device, sizeof(int) * T_VEC);
//3 - TRANSFERIR DATOS DEL HOST AL DEVICE
cudaMemcpy(v1_device, v1_host, sizeof(int) * T_VEC,
cudaMemcpyHostToDevice); //transferir el valor del host al device
cudaMemcpy(v2_device, v2_host, sizeof(int) * T_VEC,
cudaMemcpyHostToDevice);
cudaMemcpy(suma_device, suma_host, sizeof(int) * T_VEC,
cudaMemcpyHostToDevice);
//4 - EJECUTAR EN UNO O MÁS KERNELS
sumaVectores<<<1, 25>>>(v1_device, v2_device, suma_device);
//lanzar el kernel
cudaDeviceSynchronize(); //espera hasta que el kernel termine
//5 - TRANSFERIR DATOS DEL DEVICE AL HOST
cudaMemcpy(suma_host, suma_device, sizeof(int) * T_VEC,
cudaMemcpyDeviceToHost); //transferir el valor del device al host
//6 - LIBERAR RECURSOS
cudaFree(v1_device); //liberar memoria del device
cudaFree(v2_device);
cudaFree(suma_device);
//MOSTRAR RESULTADO
for (int i = 0; i < T_VEC; i++) {
std::cout << suma_host[i] << " ";
}
std::cout << "\n";
return 0;
}

Índice de hilo dentro del bloque: 0
Índice de hilo dentro del bloque: 1
Índice de hilo dentro del bloque: 2
Índice de hilo dentro del bloque: 3
Índice de hilo dentro del bloque: 4
Índice de hilo dentro del bloque: 5
Índice de hilo dentro del bloque: 6
Índice de hilo dentro del bloque: 7
Índice de hilo dentro del bloque: 8
Índice de hilo dentro del bloque: 9
Índice de hilo dentro del bloque: 10
Índice de hilo dentro del bloque: 11
Índice de hilo dentro del bloque: 12
Índice de hilo dentro del bloque: 13
Índice de hilo dentro del bloque: 14
Índice de hilo dentro del bloque: 15
Índice de hilo dentro del bloque: 16
Índice de hilo dentro del bloque: 17
Índice de hilo dentro del bloque: 18
Índice de hilo dentro del bloque: 19
Índice de hilo dentro del bloque: 20
Índice de hilo dentro del bloque: 21
Índice de hilo dentro del bloque: 22
Índice de hilo dentro del bloque: 23
Índice de hilo dentro del bloque: 24
Índice del bloque: 0
Índice del bloque: 0
Índice del bloque: 0
Índice del bloque: 0
Í

In [13]:
%%cu
#include <iostream>
#include <cuda_runtime.h>
#define T_VEC 200
//DEFINICIÓN KERNEL CUDA
__global__ void sumaVectores(int* v1_device, int* v2_device, int*
suma_device) {

printf("Índice del bloque: %d\n", blockIdx.x);
printf("Índice de hilo dentro del bloque: %d\n", (threadIdx.x+(blockIdx.x*20)));
//como en este caso hay 10 bloques de 20 hilos no es necesario hacer ningún bloque para que se haga las 200 veces
//simplemente hay que hacer la suma y el indice en el que se guarda es el id del thread + el id del bloque* 20 ya que cada bloque tiene 20 threads

suma_device[threadIdx.x+(blockIdx.x*20)] = v1_device[threadIdx.x+(blockIdx.x*20)] + v2_device[threadIdx.x+(blockIdx.x*20)];

}
//FUNCIÓN DEL HOST
int main() {
//1 - INICIALIZAR DATOS DEL HOST
int v1_host[T_VEC], v2_host[T_VEC], suma_host[T_VEC];
for (int i = 0; i < T_VEC; i++) {
v1_host[i] = 1;
v2_host[i] = 2;
}
//2 - DECLARAR Y ASIGNAR MEMORIA PARA EL DEVICE
int *v1_device, *v2_device, *suma_device; //punteros al device
cudaMalloc((void**)&v1_device, sizeof(int) * T_VEC); //asignar memoria para el device
cudaMalloc((void**)&v2_device, sizeof(int) * T_VEC);
cudaMalloc((void**)&suma_device, sizeof(int) * T_VEC);
//3 - TRANSFERIR DATOS DEL HOST AL DEVICE
cudaMemcpy(v1_device, v1_host, sizeof(int) * T_VEC,
cudaMemcpyHostToDevice); //transferir el valor del host al device
cudaMemcpy(v2_device, v2_host, sizeof(int) * T_VEC,
cudaMemcpyHostToDevice);
cudaMemcpy(suma_device, suma_host, sizeof(int) * T_VEC,
cudaMemcpyHostToDevice);
//4 - EJECUTAR EN UNO O MÁS KERNELS
sumaVectores<<<10, 20>>>(v1_device, v2_device, suma_device);
//lanzar el kernel
cudaDeviceSynchronize(); //espera hasta que el kernel termine
//5 - TRANSFERIR DATOS DEL DEVICE AL HOST
cudaMemcpy(suma_host, suma_device, sizeof(int) * T_VEC,
cudaMemcpyDeviceToHost); //transferir el valor del device al host
//6 - LIBERAR RECURSOS
cudaFree(v1_device); //liberar memoria del device
cudaFree(v2_device);
cudaFree(suma_device);
//MOSTRAR RESULTADO
for (int i = 0; i < T_VEC; i++) {
std::cout << suma_host[i] << " ";
}
std::cout << "\n";
return 0;
}

Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 4
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 9
Índice del bloque: 1
Índice del bloque: 1
Índice del bloque: 1
Índice del bloque: 1
Índice del bloque: 1
Índice del bloque: 1
Índice del bloque: 1
Índice del bl